# Data Transformation

Libraries

In [ ]:
import pandas as pd

### Loading Data

In [45]:
df = pd.read_parquet("./data/data.parquet")

#### Converting original Columns
Skip if completed

In [17]:
df = pd.read_parquet("./data/data.parquet")
columns = df.columns.to_list()

In [ ]:
def normalize_column_names(column_names):
    """
    Normalizes a list of column names by removing excessive signs, shortening names, and using underscores.

    Args:
        column_names: A list of strings representing column names.

    Returns:
        A list of normalized column names.
    """
    normalized_names = []
    for name in column_names:
        name = name.upper()
        name = name.replace("(", "").replace(")", "").replace(".", "")  # Remove parentheses and periods
        name = name.replace("(", "").replace(")", "").replace(".", "")  # Remove parentheses and periods
        name = name.replace(" ", "_")  # Replace spaces with underscores
        name = name.replace("__", "_") # Remove double underscores, if any.
        name = name.replace("ADJUSTED_", "") # Remove adjusted_ prefix
        
        name = name.replace("DELIVERY_NOTE", "DELIVERY_NO")
        name = name.replace("DELIVERY_DATE", "DELIVERY_DT")
        name = name.replace("INVENTORY_NAME", "INVENTORY")
        name = name.replace("CATALOG_NAME", "CATALOG")
        # name = name.replace("REVENUE_VAT_EXCL", "REVENUE")
        name = name.replace("TOTAL_AVG_COST", "AD_AVG_COST")
        name = name.replace("FRONT_MARGIN", "AD_FR_MARGIN")
        name = name.replace("ITEM_GROUP_ID", "ITEM_GROUP")

        
        normalized_names.append(name)
    return normalized_names

# Example usage:
column_names = [
    'DN_DELIVERY_DT', 'DELIVERY_NO', 'COUNTRY', 'TERRITORY',
    'ITEM_GROUP_ID', 'INVENTORY', 'CATALOG',
    'REVENUE_VAT_EXCL', 'ADJUSTED_TOTAL_AVG_COST',
    'AD_FR_MARGIN', 'AD_FR_MARGIN%'
]

normalized_column_names = normalize_column_names(columns)
print(normalized_column_names)

['DN_DELIVERY_DT', 'DELIVERY_NO', 'COUNTRY', 'TERRITORY', 'ITEM_GROUP', 'INVENTORY', 'CATALOG', 'REVENUE_VAT_EXCL', 'AD_AVG_COST', 'AD_FR_MARGIN', 'AD_FR_MARGIN%']


### Data Transformation

In [47]:
df.columns = ['DN_DELIVERY_DT', 'DELIVERY_NO', 'COUNTRY', 'TERRITORY', 'ITEM_GROUP', 'INVENTORY', 'CATALOG', 'REVENUE_VAT_EXCL', 'AD_AVG_COST', 'AD_FR_MARGIN', 'AD_FR_MARGIN%']
df.to_parquet("./data/clean_data.csv")

In [41]:
# Helper functions
def calculate_mode(series):
    return series.mode().iloc[0] if not series.mode().empty else None

def condition_abc_margin(x):
    if x <= 0.03:
        return 'C'
    elif x > 0.03 and x <= 0.07:
        return 'B'
    else:
        return 'A'

def condition_abc_rev(x):
    if x > 0 and x <= 0.80:
        return "A"
    elif x > 0.80 and x <= 0.95:
        return "B"
    else:
        return 'C'

def condition_xyz(x):
    if x <= 0.5:
        return 'X'
    elif x > 0.5 and x <= 1:
        return 'Y'
    else:
        return 'Z'

In [42]:
# ABC Analysis
def abc_analysis(df):
    abc = df[['TERRITORY', 'DELIVERY_NO', 'CATALOG', 'INVENTORY', 'REVENUE_VAT_EXCL', 
              'AD_FR_MARGIN', 'AD_FR_MARGIN%']]
    
    # ABC Margin Analysis
    abc_df_mode_with_ter = abc.groupby(['TERRITORY', 'INVENTORY']).agg(
        Product_Margin=('AD_FR_MARGIN%', calculate_mode),  
        Revenue=('REVENUE_VAT_EXCL', 'sum')
    ).reset_index()
    abc_df_mode_with_ter = abc_df_mode_with_ter.rename(columns={"Product_Margin":"PRODUCT_MARGIN"})
    abc_df_mode_with_ter['ABC_MARGIN_MODE'] = abc_df_mode_with_ter['PRODUCT_MARGIN'].apply(condition_abc_margin)

    # ABC Revenue Analysis
    abc_rev_with_ter = abc.groupby(['TERRITORY', 'INVENTORY']).agg(
        TOTAL_REVENUE=('REVENUE_VAT_EXCL', 'sum')).reset_index()
    
    abc_rev_with_ter['TOTAL_REVENUE_sum'] = abc_rev_with_ter.groupby('TERRITORY')['TOTAL_REVENUE'].transform('sum')
    abc_rev_with_ter = abc_rev_with_ter.sort_values(by=['TERRITORY', 'TOTAL_REVENUE'], ascending=[True, False])
    abc_rev_with_ter['CUMULATIVE_SUM'] = abc_rev_with_ter.groupby('TERRITORY')['TOTAL_REVENUE'].cumsum()
    abc_rev_with_ter['SKU_REV%'] = abc_rev_with_ter['CUMULATIVE_SUM'] / abc_rev_with_ter['TOTAL_REVENUE_sum']
    abc_rev_with_ter['ABC_REVENUE'] = abc_rev_with_ter['SKU_REV%'].apply(condition_abc_rev)

    # Merge ABC Revenue and Margin datasets
    final_ter = abc_rev_with_ter.merge(abc_df_mode_with_ter, on=['TERRITORY', 'INVENTORY'], how='left')
    final_ter['ABC(REV-MAR)'] = final_ter['ABC_REVENUE'].astype(str) + final_ter['ABC_MARGIN_MODE'].astype(str)

    return final_ter[['TERRITORY', 'INVENTORY', 'TOTAL_REVENUE', 'ABC_REVENUE', 'PRODUCT_MARGIN', 
                      'ABC_MARGIN_MODE', 'ABC(REV-MAR)']]

# XYZ Analysis
def xyz_analysis(df):
    xyz = df[['TERRITORY', 'DN_DELIVERY_DT', 'INVENTORY', 'REVENUE_VAT_EXCL']]
    xyz['YEAR'] = xyz['DN_DELIVERY_DT'].dt.year
    xyz['MONTH'] = xyz['DN_DELIVERY_DT'].dt.month
    xyz['YEAR_MONTH'] = xyz['MONTH'].astype(str) + '-' + xyz['YEAR'].astype(str)

    xyz_df_ter = xyz.groupby(['TERRITORY', 'INVENTORY', 'YEAR_MONTH'])['REVENUE_VAT_EXCL'].sum().reset_index()
    xyz_df_ter = xyz_df_ter.pivot(index=['TERRITORY', 'INVENTORY'], columns='YEAR_MONTH', 
                                   values='REVENUE_VAT_EXCL').reset_index().fillna(0)

    # Identify sales columns dynamically
    sales_columns = [col for col in xyz_df_ter.columns if col not in ['TERRITORY', 'INVENTORY']]

    # Calculate total revenue, average revenue, and standard deviation dynamically
    xyz_df_ter['TOTAL_REVENUE'] = xyz_df_ter[sales_columns].sum(axis=1, numeric_only=True)
    num_months = len(sales_columns)
    xyz_df_ter['AVERAGE_REVENUE'] = xyz_df_ter['TOTAL_REVENUE'] / num_months if num_months > 0 else 0
    xyz_df_ter['STD_DEV'] = xyz_df_ter[sales_columns].std(axis=1, numeric_only=True)

    # Calculate Coefficient of Variance (CoV)
    xyz_df_ter['CoV'] = xyz_df_ter['STD_DEV'] / xyz_df_ter['AVERAGE_REVENUE']
    xyz_df_ter['TERRITORY_XYZ'] = xyz_df_ter['CoV'].apply(condition_xyz)

    return xyz_df_ter[['TERRITORY', 'INVENTORY', 'TOTAL_REVENUE', 'CoV', 'STD_DEV', 'AVERAGE_REVENUE', 'TERRITORY_XYZ']]

# ABC-XYZ Merging
def merge_abc_xyz(territory_abc_df, xyz_df):
    abc_xyz = territory_abc_df.merge(xyz_df, on=['TERRITORY', 'INVENTORY'], how='left')
    abc_xyz['ABC_XYZ'] = abc_xyz['ABC(REV-MAR)'].astype(str) + abc_xyz['TERRITORY_XYZ'].astype(str)
    return abc_xyz

# Main function to execute all analyses
def main(filepath):
    # df = load_and_preprocess_data(filepath)
    territory_abc_df = abc_analysis(df)
    xyz_df = xyz_analysis(df)
    abc_xyz = merge_abc_xyz(territory_abc_df, xyz_df)
    abc_xyz = abc_xyz.rename(columns={"TOTAL_REVENUE_x":"TOTAL_REVENUE_X", "TOTAL_REVENUE_y":"TOTAL_REVENUE_Y"})
    
    return abc_xyz

### Test

In [43]:
main(df).head()

/tmp/ipykernel_1234/1992248013.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xyz['YEAR'] = xyz['DN_DELIVERY_DT'].dt.year
/tmp/ipykernel_1234/1992248013.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xyz['MONTH'] = xyz['DN_DELIVERY_DT'].dt.month
/tmp/ipykernel_1234/1992248013.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pan

,TERRITORY,INVENTORY,TOTAL_REVENUE_X,ABC_REVENUE,PRODUCT_MARGIN,ABC_MARGIN_MODE,ABC(REV-MAR),TOTAL_REVENUE_Y,CoV,STD_DEV,AVERAGE_REVENUE,TERRITORY_XYZ,ABC_XYZ
0,Abuja,Devon King's Cooking Oil 25L,88439380.00,A,0.047619,B,AB,88439380.00,1.484586,1.193599e+07,8.039944e+06,Z,ABZ
1,Abuja,Golden Penny Spaghetti 500g,84236614.12,A,0.089904,A,AA,84236614.12,1.328130,1.017065e+07,7.657874e+06,Z,AAZ
2,Abuja,IRS FLOUR 50KG,80921700.00,A,0.023376,C,AC,80921700.00,1.585146,1.166116e+07,7.356518e+06,Z,ACZ
3,Abuja,Devon King's Cooking Oil 5L,72945550.00,A,0.040000,B,AB,72945550.00,1.301417,8.630235e+06,6.631414e+06,Z,ABZ
4,Abuja,Maltina Malt Can 33cl*24,67446465.19,A,0.009334,C,AC,67446465.19,0.882144,5.408862e+06,6.131497e+06,Y,ACY
